In [1]:
from src.models.sequence.hyena_seq import FNOBaseline
from src.dataloaders.icl_pde import PDEDataModule
import torch

In [2]:
dataset = PDEDataModule(
    num_examples=64,
    num_test_examples=64,
    num_initial_conditions=20,
    pde="1d_burgers_icl_t2",
    data_dir="/usr/xtmp/jwl50/PDEBench/data/",
    file_name="1D/Burgers/Train/1D_Burgers_Sols_Nu0.1.hdf5",
)
dataset.setup()
data_train = dataset.train_dataloader()

Reading file:
Read file
Train tensor shape: torch.Size([64, 201, 1024])
ICL train examples shape: torch.Size([64, 40, 1024])


In [3]:
for data in data_train:
    x,y,z = data
    break

In [4]:
print(f"x: {x.shape}")
print(f"y: {y.shape}")
print(f"z: {z.shape}")

x: torch.Size([32, 39, 1024])
y: torch.Size([32, 39, 1024])
z: torch.Size([32])


In [9]:
# Verify that x[:, 1:, :] = y[:, :-1, :]
torch.all(x[:, 1:, :] == y[:, :-1, :])

tensor(True)

In [5]:
model = FNOBaseline(
    d_model=1024,
    dropout=0.1,
)

In [6]:
pos = model.positional(z)
print(f"pos: {pos.shape}")
print(pos.device)

pos: torch.Size([32, 1024])
cuda:0


In [7]:
model.to("cuda")
f_out, _ = model(x.cuda(), z.cuda())
print(f"f_out: {f_out.shape}")

f_out: torch.Size([32, 20, 1024])


In [10]:
c = torch.cumsum(f_out, dim=1)
print(f"c: {c.shape}")

c: torch.Size([32, 20, 1024])


In [16]:
c2 = torch.arange(1, c.shape[1]+1, device=c.device)
c / c2[None, :, None]

tensor([[[-0.0341, -0.0668, -0.0340,  ..., -0.0467, -0.0352, -0.0471],
         [-0.0339, -0.0705, -0.0339,  ..., -0.0459, -0.0359, -0.0462],
         [-0.0339, -0.0720, -0.0339,  ..., -0.0455, -0.0363, -0.0459],
         ...,
         [-0.0407, -0.0847, -0.0406,  ..., -0.0558, -0.0451, -0.0563],
         [-0.0405, -0.0862, -0.0404,  ..., -0.0550, -0.0452, -0.0555],
         [-0.0411, -0.0906, -0.0410,  ..., -0.0540, -0.0468, -0.0545]],

        [[-0.0370, -0.1131, -0.0370,  ..., -0.1041, -0.0370, -0.1065],
         [-0.0490, -0.1565, -0.0490,  ..., -0.1455, -0.0490, -0.1486],
         [-0.1095, -0.2426, -0.1098,  ..., -0.2315, -0.1094, -0.2347],
         ...,
         [-0.0636, -0.1569, -0.0638,  ..., -0.1467, -0.0634, -0.1500],
         [-0.0626, -0.1504, -0.0628,  ..., -0.1407, -0.0625, -0.1439],
         [-0.0615, -0.1447, -0.0618,  ..., -0.1355, -0.0614, -0.1385]],

        [[-0.0371, -0.1133, -0.0371,  ..., -0.0348, -0.0396, -0.0344],
         [-0.0361, -0.0855, -0.0360,  ..., -0

In [ ]:
import math
d_model=1024
r = torch.arange(1, d_model//2+1)
temp = (-math.log(10000.0) / r)
z2 = z.unsqueeze(-1).repeat(*([1]*len(z.shape)), d_model//2).float()
div_term = z.unsqueeze(-1).repeat(*([1]*len(z.shape)), d_model//2).float()
div_term[..., :] *= temp
div_term = torch.exp(div_term)
pe = torch.zeros(*z.shape, d_model)
pe[..., 0::2] = torch.sin(z2 * div_term)
pe[..., 1::2] = torch.cos(z2 * div_term)
# pe[, :] *= z
pe
# temp.repeat(*z.shape, 1)
# print(temp.shape)
# print((z*temp).shape)
# div_term = torch.exp(z * (-math.log(10000.0) / (d_model/2)))
# print(div_term.shape)
# print(z.shape)
# pe = torch.zeros(*z.shape, d_model)
# print(pe.shape)
# pe[..., 0::2] = torch.sin(z * div_term)
# pe[..., 1::2] = torch.cos(z * div_term)